In [1]:
import soccerdata as sd
import pandas as pd
from flask import Flask, jsonify, Response
import json
from flask import Flask, request, Response
import requests
import numpy as np

app = Flask(__name__)

df = pd.read_csv('Merged_players_Whoscored.csv')

# Create a dictionary of player names and their ratings
player_ratings = df.set_index('Name')['Rating'].to_dict()
def get_top_10_players(league_code, season=2021):
    fbref = sd.FBref(leagues=league_code, seasons=season)
    player_season_stats = fbref.read_player_season_stats(stat_type="standard")
    player_season_stats = player_season_stats.reset_index(level=("league", "season", "team", "player"), col_level=0)
    return player_season_stats.head(10)

top_10_it = get_top_10_players("ITA-Serie A")
top_10_eng = get_top_10_players("ENG-Premier League")
top_10_bundesliga = get_top_10_players("GER-Bundesliga")
top_10_laliga = get_top_10_players("ESP-La Liga")
top_10_ligue1 = get_top_10_players("FRA-Ligue 1")

# Merge the dataframes
merged_data = pd.concat([top_10_it, top_10_eng, top_10_bundesliga, top_10_laliga, top_10_ligue1], axis=0, ignore_index=True)



player_ratings = df.set_index('Name')['Rating'].to_dict()



merged_data['rating'] = merged_data['player'].map(player_ratings)
merged_data['rating'] = merged_data['rating'].fillna(np.round(np.random.uniform(5, 10)))
merged_data['rating'] = merged_data['rating'] * 10
merged_data['rating'] = merged_data['rating'].apply(lambda x: int(x))


datap = ['https://media-1.api-sports.io/football/players/484.png',
        'https://media-2.api-sports.io/football/players/30421.png',
        'https://media-1.api-sports.io/football/players/14316.png',
        'https://media-2.api-sports.io/football/players/884.png',
        None,
        None,
        'https://media-1.api-sports.io/football/players/2495.png',
        'https://media-3.api-sports.io/football/players/30747.png',
        'https://media-1.api-sports.io/football/players/30423.png',
        'https://media-1.api-sports.io/football/players/1456.png',
        'https://media-3.api-sports.io/football/players/1467.png',
        'https://media-1.api-sports.io/football/players/1438.png',
        'https://media-2.api-sports.io/football/players/1460.png',
        'https://media-3.api-sports.io/football/players/19016.png',
        'https://media-1.api-sports.io/football/players/190.png',
        None,
        'https://media-2.api-sports.io/football/players/47522.png',
        'https://media-2.api-sports.io/football/players/1468.png',
        'https://media-2.api-sports.io/football/players/1161.png',
        'https://media-1.api-sports.io/football/players/25011.png',
        'https://media-2.api-sports.io/football/players/25010.png',
        'https://media-1.api-sports.io/football/players/25030.png',
        'https://media-3.api-sports.io/football/players/25355.png',
        None,
        None,
        'https://media-1.api-sports.io/football/players/7682.png',
        'https://media-2.api-sports.io/football/players/25007.png',
        'https://media-2.api-sports.io/football/players/1144.png',
        'https://media-2.api-sports.io/football/players/25499.png',
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        'https://media-2.api-sports.io/football/players/3234.png',
        'https://media-1.api-sports.io/football/players/21146.png',
        'https://media-1.api-sports.io/football/players/21446.png',
        'https://media-2.api-sports.io/football/players/21153.png',
        'https://media-2.api-sports.io/football/players/21652.png',
        'https://media-1.api-sports.io/football/players/20656.png',
        'https://media-1.api-sports.io/football/players/3177.png',
        'https://media-2.api-sports.io/football/players/2049.png',
        'https://media-2.api-sports.io/football/players/3237.png',
        'https://media-2.api-sports.io/football/players/20633.png'
]
        
dp = pd.DataFrame(datap, columns=['p'])

merged_data = pd.concat([merged_data, dp], axis=1)

merged_data.columns = [col[0] + ' ' + col[1] if len(col) > 1 and col[1] != '' else col[0] for col in merged_data.columns]



def get_leagues():
    url = "https://api-football-v1.p.rapidapi.com/v3/leagues"
    headers = {
        "X-RapidAPI-Key": "06b1cce7b7mshd1d68e8b760a40cp1e5a88jsnced3494ddea2",
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }
    response = requests.request("GET", url, headers=headers)
    data = response.json()
    leagues = []
    for league in data['response']:
        league_id = league['league']['id']
        league_name = league['league']['name']
        leagues.append({'id': league_id, 'name': league_name})
    return leagues

def get_league_id(league_name_in_dataframe):
    leagues = get_leagues()
    league_name_to_search = league_name_in_dataframe.split('-')[1]
    league_ids = []
    for league in leagues:
        if league['name'] == league_name_to_search:
            league_ids.append(league['id'])
    return league_ids


@app.route('/')
def hello_world():
    return 'Hello, World!'

@app.route('/api')
def api():
    return {'message': 'This is an API endpoint.'}


@app.route('/data')
def display_data():
    json_data = merged_data.to_json(orient='records')
    data = json.loads(json_data)
    formatted_json = json.dumps(data, indent=4, ensure_ascii=False)
    return Response(formatted_json, content_type='application/json')

@app.route('/data/<player_name>')
def display_data_sibgle(player_name):
    player_data = merged_data[merged_data['player'] == player_name]
    json_data = player_data.to_json(orient='records')
    data = json.loads(json_data)
    formatted_json = json.dumps(data, indent=4, ensure_ascii=False)

    return Response(formatted_json, content_type='application/json')

if __name__ == "__main__":
    app.run(port=5000)


[03/31/23 00:09:52] INFO     No custom team name replacements found. You can configure these in       _config.py:85
                             C:\Users\Dhiaeddine Ayeb\soccerdata\config\teamname_replacements.json.                

                    INFO     No custom league dict found. You can configure additional leagues in    _config.py:159
                             C:\Users\Dhiaeddine Ayeb\soccerdata\config\league_dict.json.                          

                    INFO     Saving cached data to C:\Users\Dhiaeddine Ayeb\soccerdata\data\FBref     _common.py:89

c:\Users\Dhiaeddine Ayeb\AppData\Local\Programs\Python\Python311\Lib\site-packages\soccerdata\_common.py:462: UserWarning: Season id "2021" is ambiguous: interpreting as "20-21"
  warnings.warn(msg)


[03/31/23 00:09:53] INFO     Saving cached data to C:\Users\Dhiaeddine Ayeb\soccerdata\data\FBref     _common.py:89

...

[03/31/23 00:44:12] INFO     127.0.0.1 - - [31/Mar/2023 00:44:12] "GET /data/Aleksei%20Miranchuk   _internal.py:224
                             HTTP/1.1" 200 -                                                                       

[03/31/23 00:44:17] INFO     127.0.0.1 - - [31/Mar/2023 00:44:17] "GET /data HTTP/1.1" 200 -       _internal.py:224

[03/31/23 00:44:25] INFO     127.0.0.1 - - [31/Mar/2023 00:44:25] "GET /data/Amad%20Diallo         _internal.py:224
                             HTTP/1.1" 200 -                                                                       

[03/31/23 00:44:30] INFO     127.0.0.1 - - [31/Mar/2023 00:44:30] "GET /data HTTP/1.1" 200 -       _internal.py:224

[03/31/23 00:44:37] INFO     127.0.0.1 - - [31/Mar/2023 00:44:37] "GET /data HTTP/1.1" 200 -       _internal.py:224